### Generating `publications.json` partitions

This is a template notebook for generating metadata on publications - most importantly, the linkage between the publication and dataset (datasets are enumerated in `datasets.json`)

Process goes as follows:
1. Import CSV with publication-dataset linkages. Your csv should have at the minimum, fields (spelled like the below):
    * `dataset` to hold the dataset_ids, and 
    * `title` for the publication title. 

Update the csv with these field names to ensure this code will run.  We read in, dedupe and format the title
2. Match to `datasets.json` -- alert if given dataset doesn't exist yet
3. Generate list of dicts with publication metadata
4. Write to a publications.json file

#### Import CSV containing publication-dataset linkages

Set `linkages_path` to the location of the csv containg dataset-publication linkages and read in csv

In [1]:
import pandas as pd
import os
import datetime

In [14]:
file_name = 'WisconsinUnemploymentInsurance_linkages_mynoa.csv'
rcm_subfolder = '20191119_WisconsinUnemploymentInsurance'
parent_folder = '/Users/mynoa/rc_project/RichContextMetadata/metadata'
linkages_path =  os.path.join(parent_folder,rcm_subfolder,file_name)
# linkages_path =  os.path.join(os.getcwd(),'SNAP_DATA_DIMENSIONS_SEARCH_DEMO.csv')
linkages_csv = pd.read_csv(linkages_path)

Format/clean linkage data - apply `scrub_unicode` to `title` field.

In [3]:
import unicodedata

In [15]:
def scrub_unicode (text):
    """
    try to handle the unicode edge cases encountered in source text,
    as best as possible
    """
    x = " ".join(map(lambda s: s.strip(), text.split("\n"))).strip()

    x = x.replace('“', '"').replace('”', '"')
    x = x.replace("‘", "'").replace("’", "'").replace("`", "'")
    x = x.replace("`` ", '"').replace("''", '"')
    x = x.replace('…', '...').replace("\\u2026", "...")
    x = x.replace("\\u00ae", "").replace("\\u2122", "")
    x = x.replace("\\u00a0", " ").replace("\\u2022", "*").replace("\\u00b7", "*")
    x = x.replace("\\u2018", "'").replace("\\u2019", "'").replace("\\u201a", "'")
    x = x.replace("\\u201c", '"').replace("\\u201d", '"')

    x = x.replace("\\u20ac", "€")
    x = x.replace("\\u2212", " - ") # minus sign

    x = x.replace("\\u00e9", "é")
    x = x.replace("\\u017c", "ż").replace("\\u015b", "ś").replace("\\u0142", "ł")    
    x = x.replace("\\u0105", "ą").replace("\\u0119", "ę").replace("\\u017a", "ź").replace("\\u00f3", "ó")

    x = x.replace("\\u2014", " - ").replace('–', '-').replace('—', ' - ')
    x = x.replace("\\u2013", " - ").replace("\\u00ad", " - ")

    x = str(unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8"))

    # some content returns text in bytes rather than as a str ?
    try:
        assert type(x).__name__ == "str"
    except AssertionError:
        print("not a string?", type(x), x)

    return x

Scrub titles of problematic characters, drop nulls and dedupe

In [16]:
linkages_csv = linkages_csv.loc[pd.notnull(linkages_csv.dataset)].drop_duplicates()
linkages_csv = linkages_csv.loc[pd.notnull(linkages_csv.title)].drop_duplicates()
linkages_csv['title'] = linkages_csv['title'].apply(scrub_unicode)

In [17]:
linkages_csv.head()

,dataset,doi,journal,search_term,title,url
0,dataset-063,10.1177/1091142116629204,Public Finance Review,Wisconsin Unemployment Insurance,EITC Use in Shared Placement Cases,https://doi.org/10.1177/1091142116629204
1,dataset-063,10.4324/9781315394503,NaN,Wisconsin Unemployment Insurance,"Federalism and the Making of America, 2nd",https://doi.org/10.4324/9781315394503
2,dataset-063,10.1177/0192513x14565701,"{'id': 'jour.1091259', 'title': 'Journal of Fa...",Wisconsin Unemployment Insurance,Child Support and Subsequent Nonmarital Fertil...,https://doi.org/10.1177/0192513x14565701
3,dataset-063,10.1016/j.childyouth.2016.10.018,"{'id': 'jour.1092724', 'title': 'Children and ...",Wisconsin Unemployment Insurance,Making parents pay: The unintended consequence...,https://doi.org/10.1016/j.childyouth.2016.10.018
4,dataset-063,10.1007/s10940-014-9242-5,"{'id': 'jour.1041809', 'title': 'Journal of Qu...",Wisconsin Unemployment Insurance,An Experimental Evaluation of a Comprehensive ...,https://doi.org/10.1007/s10940-014-9242-5


In [18]:
pub_metadata_fields = ['title']
original_metadata_cols = list(set(linkages_csv.columns.values.tolist()) - set(pub_metadata_fields)-set(['dataset']))

#### Generate list of dicts of metadata

Read in `datasets.json`. Update `datasets_path` to your local.

In [19]:
import json

In [20]:
datasets_path = '/Users/mynoa/rc_project/RCDatasets/datasets.json'

with open(datasets_path) as json_file:
    datasets = json.load(json_file)

Create list of dictionaries of publication metadata. `format_metadata` iterrates through `linkages_csv` dataframe, splits the `dataset` field (for when multiple datasets are listed); throws an error if the dataset doesn't exist and needs to be added to `datasets.json`.

In [21]:
def create_pub_dict(linkages_dataframe,datasets):
    pub_dict_list = []
    for i, r in linkages_dataframe.iterrows():
        r['title'] = scrub_unicode(r['title'])
        ds_id_list = [f for f in [d.strip() for d in r['dataset'].split(",")] if f not in [""," "]]
        for ds in ds_id_list:
            check_ds = [b for b in datasets if b['id'] == ds]
            if len(check_ds) == 0:
                print('dataset {} isnt listed in datasets.json. Please add to file'.format(ds))
        required_metadata = r[pub_metadata_fields].to_dict()
        required_metadata.update({'datasets':ds_id_list})
        pub_dict = required_metadata
        if len(original_metadata_cols) > 0:
            original_metadata = r[original_metadata_cols].to_dict()
            original_metadata.update({'date_added':datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')})
            pub_dict.update({'original':original_metadata})
        pub_dict_list.append(pub_dict)
    return pub_dict_list

Generate publication metadata and export to json

In [22]:
linkage_list = create_pub_dict(linkages_csv,datasets)

Update `pub_path` to be: 
`<name_of_subfolder>_publications.json`

In [23]:
json_pub_path = os.path.join('/Users/mynoa/rc_project/RCPublications/partitions/',rcm_subfolder+'_publications.json')

In [24]:
with open(json_pub_path, 'w') as outfile:
    json.dump(linkage_list, outfile, indent=2)